# "Record" simulated neural activity from basic models (NA, LN, LINEAR)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import warnings; warnings.filterwarnings("ignore")

from aux import load_npy
from record_0_main import smlt_ma, smlt_linma, smlt_lnma
from record_2_lin_ln import smlt_lnma_tweaked

cc = np.concatenate

FPS_DN = 8.5  # sampling rate from neural data
DT_DN = 1/FPS_DN

FPS = 30.03  # sampling rate of behavioral data
DT = 1/FPS

Load either Coen or WT Gold behavior data

In [2]:
df_behav = pd.read_csv('data/simple/c_song_f_behav.csv')
CTR_OFFSET = 0

# df_behav = pd.read_csv('data/simple/w_song_f_behav.csv')
# CTR_OFFSET = 276

# df_behav = pd.read_csv('data/simple/block_song_f_behav.csv')
# CTR_OFFSET = 998

df_behav

,ID,FRAME,T,Q,S,P,F,FFV,FFA,FLS,FRS,FMFV,FMLS
0,0,0,0.000000,1,0,0,0,0.231,0.0000,0.159,0.077,-0.058526,0.274257
1,0,1,0.033300,1,0,0,0,0.314,0.0385,0.025,0.044,0.100623,0.298490
2,0,2,0.066600,1,0,0,0,0.308,0.0025,0.029,0.011,0.147629,0.271865
3,0,3,0.099900,1,0,0,0,0.319,0.0020,0.016,0.016,0.091174,0.306112
4,0,4,0.133200,1,0,0,0,0.312,-0.0315,0.030,0.002,0.065871,0.306439
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4978565,275,27100,902.430902,1,0,0,0,0.217,0.0280,0.029,0.058,-0.171418,0.136184
4978566,275,27101,902.464202,1,0,0,0,0.210,-0.1045,0.138,0.322,0.032522,0.249171
4978567,275,27102,902.497502,1,0,0,0,0.008,-0.0910,0.034,0.260,0.033202,0.010844
4978568,275,27103,902.530803,1,0,0,0,0.028,0.0695,0.010,0.090,0.012059,0.027177


In [3]:
# split big df into dfs for individual trials
ntr = np.max(df_behav.ID) + 1
dfs_tr = [df_behav[df_behav.ID == i] for i in range(ntr)]

tr_lens = np.array([len(df_tr) for df_tr in dfs_tr])
tr_lens_cum = cc([[0], np.cumsum(tr_lens)])

n_t_total = np.sum(tr_lens)
assert n_t_total == tr_lens_cum[-1]
df_behav = None

# NA
Nonlinear accumulation dynamical system neural responses

In [5]:
SAVE_DIR = 'data/simple/mlv/neur_basic/baker_ma'
PFX = 'mlv_baker_ma'
smlt = smlt_ma  # simulation function for MA neurons

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)
    
# neural response fits from Baker data
params = load_npy('data/simple/neur/baker_ma.npy')
expt_ids = params['EXPT_ID']

In [6]:
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    # simulate population response
    rs = smlt(i_s, i_p, params, DT)
        
    df_tr_nrl[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
        
    np.save(os.path.join(SAVE_DIR, f'{PFX}_tr_{ctr+CTR_OFFSET}.npy'), np.array([{'df': df_tr_nrl}]))

....................................................................................................................................................................................................................................................................................

# LIN-NA

Linear filters matched to NA fits.

In [7]:
SAVE_DIR = 'data/simple/mlv/neur_basic/baker_linma'
PFX = 'mlv_baker_linma'
smlt = smlt_linma

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)
    
# ma response fits from Baker data
params = load_npy('data/simple/neur/baker_ma.npy')
expt_ids = params['EXPT_ID']

In [8]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    rs = smlt(i_s, i_p, params, DT)
        
    df_tr_nrl[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
    
    np.save(os.path.join(SAVE_DIR, f'{PFX}_tr_{ctr+CTR_OFFSET}.npy'), np.array([{'df': df_tr_nrl}]))

....................................................................................................................................................................................................................................................................................

# LN-NA

Linear-nonlinear RELU neurons with filters matched to NA fits.

In [4]:
SAVE_DIR = 'data/simple/mlv/neur_basic/baker_lnma'
PFX = 'mlv_baker_lnma'
smlt = smlt_lnma

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)
    
# ma response fits from Baker data
params = load_npy('data/simple/neur/baker_ma.npy')
expt_ids = params['EXPT_ID']

In [5]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    rs = smlt(i_s, i_p, params, DT)
        
    df_tr_nrl[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
    
    np.save(os.path.join(SAVE_DIR, f'{PFX}_tr_{ctr+CTR_OFFSET}.npy'), np.array([{'df': df_tr_nrl}]))

....................................................................................................................................................................................................................................................................................

# LN-NA-TWEAKED

Linear-nonlinear RELU neurons with double-exp filters derived from NA fits then adjusted

In [6]:
SAVE_DIR = 'data/simple/mlv/neur_basic/baker_lnma_tweaked'
PFX = 'mlv_baker_lnma_tweaked'
smlt = smlt_lnma_tweaked

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)
    
# ma response fits from Baker data
params = load_npy('data/simple/neur/baker_lnma_tweaked.npy')
expt_ids = params['EXPT_ID']

In [7]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    rs = smlt(i_s, i_p, params, DT)
        
    df_tr_nrl[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
    
    np.save(os.path.join(SAVE_DIR, f'{PFX}_tr_{ctr+CTR_OFFSET}.npy'), np.array([{'df': df_tr_nrl}]))

....................................................................................................................................................................................................................................................................................

# LN-NA-TWEAKED_4

Linear-nonlinear RELU neurons with double-exp filters derived from NA fits then the 4 NA-matched params adjusted

In [4]:
SAVE_DIR = 'data/simple/mlv/neur_basic/baker_lnma_tweaked_4'
PFX = 'mlv_baker_lnma_tweaked_4'
smlt = smlt_lnma

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)
    
# ma response fits from Baker data
params = load_npy('data/simple/neur/baker_lnma_tweaked_4.npy')
expt_ids = params['EXPT_ID']

In [5]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(float)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
    
    rs = smlt(i_s, i_p, params, DT)
        
    df_tr_nrl[[f'R_{expt_id}' for expt_id in expt_ids]] = rs
    
    np.save(os.path.join(SAVE_DIR, f'{PFX}_tr_{ctr+CTR_OFFSET}.npy'), np.array([{'df': df_tr_nrl}]))

....................................................................................................................................................................................................................................................................................